Demonstrate frequency scanning using a simulated resonator.  

* kids.sweep() uses a single tone
* kids.set_tones() and kids.sweep_tones() first sets tones at nominal values, and the sweeps them all, using two tones:  one centered on the resonator and a second tone, off resonance.

These connections use "Dual 1" and "Sim 1" through 1-4 GHz Baluns

* DAC 2_230 --> 1-4 GHz Baluns --> ADC 2_227
* DAC 2_231 --> 1-4 GHz Baluns --> ADC 2_226

In [ ]:
import sys
sys.path.append('../../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
board = Scan.getBoard()
if board == 'zcu111':
    bitfile = 'mkids_2x2_kidsim_v1'    
elif board == 'zcu216':
    bitfile = 'mkids_2x2_kidsim_v2'
else:
    raise Exception("Do not know what to do with %s"%board)
scan = Scan.Scan(bitfile, iKids=1, iSimu=1)

In [ ]:
fMixerSimu = 3512
fMixerKids = 3700
fResonance = 3789.0123
fOffset = 12.3456
fc = 3790
df = 5 # Sweep Width
N = 250 # Number of points in a sweep

In [ ]:
# Initialize simulation chain with one resonance
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()

# Set mixer.
simu.set_mixer_frequency(fMixerSimu)

# Put one resonator 
print("fResonance = {} MHz".format(fResonance))
simu.enable(fResonance)

In [ ]:
# Get the kids chain
kids = scan.kidsChain
qoutAnalysis = 1
qoutSynthesis = 1
kids.analysis.qout(qoutAnalysis)
kids.synthesis.qout(qoutSynthesis)


In [ ]:

f,a,phi=kids.sweep(fc-df/2,fc+df/2,N=N,g=0.9, showProgress=False, doProgress=True)

In [ ]:
#plt.plot(f,20*np.log10(a/max(a)))
plt.plot(f,a)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude ADUs");
plt.axvline(fResonance, c='r')

In [ ]:
#raise Exception()

In [ ]:
plt.plot(f,phi, '.-')
plt.xlabel("frequency (MHz)")
plt.ylabel("phase (Rad)")

In [ ]:
kids.set_mixer_frequency(fMixerKids)

freqs = np.array([fResonance, fResonance+fOffset])
fis = np.array([0.1, 0.2])
gs = np.array([0.4, 0.6])
kids.set_tones(freqs, fis, gs)
xs = kids.sweep_tones(df, N, mean=True, doProgress=True)



In [ ]:
offsets = kids.get_sweep_offsets(df, N)

for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(offsets, np.abs(xs[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Amplitude (ADUs)")
plt.title("Resonance at %f"%fResonance)
plt.legend()


In [ ]:
# Repeat the scan
kids.set_tones(freqs, fis, gs)
xs2 = kids.sweep_tones(df, N, mean=True, doProgress=True)
for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(offsets, np.abs(xs2[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Amplitude (ADUs)")
plt.title("Resonance at %f"%fResonance)
plt.legend()


In [ ]:
# Plot difference of first and second scan
for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(offsets, np.abs(xs2[:,iTone])-np.abs(xs[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Delta Amplitude (ADUs)")
plt.legend()


In [ ]:
# Now turn off simulated resonance and scan again
simu.alloff()
kids.set_tones(freqs, fis, gs)
xs3 = kids.sweep_tones(df, N, mean=True, doProgress=True)


In [ ]:
for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(offsets, np.abs(xs3[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Amplitude (ADUs)")
plt.legend()
_ = plt.title("No resonance")


In [ ]:
for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(offsets, np.angle(xs3[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Phase (Rad)")
plt.legend()
_ = plt.title("No resonance")


In [ ]:
delayFreqs = scan.inCh2Freq([9,10])
kids.set_tones(delayFreqs, fis, gs)
delayDf = .2
delayN = 20
delayOffsets = kids.get_sweep_offsets(delayDf, delayN)

xs4 = kids.sweep_tones(delayDf, delayN, mean=True, doProgress=True)


In [ ]:
for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(delayOffsets, np.angle(xs4[:,iTone]), "-o", label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Phase (Rad)")
plt.legend()
_ = plt.title("No resonance")


In [ ]:
scan.inFreq2ch(freqs)

In [ ]:
delay = measureDelay(delayOffsets, xs4[:,0], plotFit=True)

In [ ]:
print(delay)

In [ ]:
# Turn the resonator back on
simu.enable(fResonance)

In [ ]:
# Scan with resonator on
freqs = np.array([fResonance, fResonance+fOffset])
fis = np.array([0.1, 0.2])
gs = np.array([0.4, 0.6])
kids.set_tones(freqs, fis, gs)
xs5 = kids.sweep_tones(df, N, mean=True, doProgress=True)



In [ ]:
for iTone,qFreq in enumerate(kids.qFreqs):
    plt.plot(offsets, np.abs(xs5[:,iTone]), label="%d %f"%(iTone,qFreq))
plt.xlabel("Frequency offset (MHz)")
plt.ylabel("Amplitude (ADUs)")
plt.legend()
_ = plt.title("Resonance at %f"%fResonance)